## Test Plan Generator 

* This project reads the Functional specification document in .pdf, .txt and .docx format and generates a comprehensive Test Plan.
* Built using Python and Gradio.

### Setup Instructions:

Create the environment: 

* conda env create -f environment.yaml
* conda activate test-plan-generator

In [ ]:
import os
from docx import Document
import pdfplumber
from openai import OpenAI
import gradio as gr
import tempfile
from fpdf import FPDF
from dotenv import load_dotenv

In [ ]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
openai = OpenAI()

In [ ]:
class FileParser:
    def __init__(self, file):
        self.file = file
        self.filename = os.path.splitext(file)[0].lower()
        self.file_extn = os.path.splitext(file)[1].lower()

    def extract_text(self):
        if self.file_extn == ".txt":
            with open(self.file.name, "r", encoding="utf-8") as f:
                return f.read()
        elif self.file_extn == ".pdf":
            with pdfplumber.open(self.file) as pdf:
                return "\n".join(page.extract_text() or "" for page in pdf.pages)
        elif self.file_extn == ".docx":
            document = Document(self.file)
            return "\n".join(para.text for para in document.paragraphs)
        else:
            raise ValueError (f"Unsupported file format: {self.file_ext}. Please upload the file in .txt, .pdf, or .docx format")

In [ ]:
class TestPlanGenerator:
    def __init__(self, model="gpt-4o-mini"):
        self.model = model
    
    def generate_prompts(self, spec_txt):
        system_prompt = """
        You are a senior QA engineer with deep expertise in analyzing functional specification documents and designing comprehensive test plans.
        
        When given a functional specification, your job is to:
        - Summarize the specification clearly under "Description" section.
        - State the test plan's goals under "Objective" section.
        - Identify all functional test cases and present them in a structured format as given below:
        
          Test Case Format:
            - Test Case ID: TC-XXX
              Title: [Brief title of the Test Case, eg: Verify user registration with email missing domain name]
              Description: [What this test case is intended to verify]
              Steps:
                1. [Step 1]
                2. [Step 2]
                ...
              Expected Result: [What should happen if the functionality works correctly]
        
        Follow this format strictly and ensure clarity, correctness, and full test coverage of the provided functional specification. 
        If the specification document is empty, clearly state that no test plan can be generated.
        """

        
        user_prompt = f"""
        Please analyze the following functional specification and generate a detailed test plan in the specified format:
        Functional Specification:
        \"\"\"
        {spec_txt}
        \"\"\"
        1. **Description**: A brief summary of the functional specification.
        2. **Objective**: The purpose and scope of the test plan.
        3. **Test Cases**: A list of functional test cases, each including:
            - **Test Case ID**: (e.g., TC-001, TC-002...)
            - **Title**: A short title describing the test case.
            - **Description**: What this test case is intended to verify.
            - **Steps**:
                1. Step one
                2. Step two
                ...
            - **Expected Result**: The expected behavior or outcome if the functionality works as intended.
        
        Ensure clarity and full functional coverage. Format the output clearly.
        """
        return system_prompt, user_prompt
    
    def generate_test_plan(self, spec_txt):
        system_prompt, user_prompt = self.generate_prompts(spec_txt)
        messages = [
            {"role" : "system" , "content" : system_prompt},
            {"role" : "user" , "content" : user_prompt}
        ]
        try: 
            response = openai.chat.completions.create(
                model = self.model,
                messages = messages,
                temperature = 0.3,
                max_tokens = 1500,
                stream= True
            )
            for chunk in response:
                result = chunk.choices[0].delta.content or ''
                yield result  
                
        except Exception as e:
            return f"LLM Error: \n{e}"

In [ ]:
class UserAppInterface:
    def __init__(self):
        self.testplan_generator = TestPlanGenerator()
    
    def generate_tp_file(self, file, output_format):
        try:
            parser = FileParser(file)
            spec_txt = parser.extract_text()
            test_plan = ""
            response = self.testplan_generator.generate_test_plan(spec_txt)
            for chunk in response:
                test_plan += chunk
                yield test_plan, None

            tp_name = parser.filename
            tp_extn = output_format.lstrip(".").lower()
            tp_path = os.path.join(tempfile.gettempdir(), f"{tp_name}.{tp_extn}")
            if tp_extn == "txt":
                with open(tp_path, 'w', encoding="utf-8") as f:
                    f.write(test_plan)
            elif tp_extn == "pdf":
                pdf = FPDF()
                pdf.add_page()
                pdf.set_auto_page_break(auto=True, margin=15)
                pdf.set_font("Arial", size=12)
                for line in test_plan.split('\n'):
                    pdf.multi_cell(0, 10, txt=line)
                pdf.output(tp_path)
            elif tp_extn == "docx":
                document = Document()
                document.add_heading('Test plan', 0)
                for line in test_plan.split('\n'):
                    document.add_paragraph(line)
                document.save(tp_path)
            else:
                return f"Unsupported file format {tp_extn}"
            yield test_plan, tp_path
        
        except Exception as e:
            yield  f"Error: {e}", None

    def launch_ui(self):
        gr.Interface(fn=self.generate_tp_file,
                     inputs=[
                         gr.File(label="Please upload the Functional Specification file in .txt, .pdf or .docx format"),
                         gr.Dropdown(choices=[".pdf", ".txt", ".docx"], value=".txt", label="Please choose the file format")
                     ],
                     outputs=[
                        gr.Textbox(autoscroll=True, show_copy_button=True, lines=20, label="Test Plan:"),
                        gr.File(label="Download:")
                     ],
                     title="Test Plan Generator:").launch()   
               

In [ ]:
UserAppInterface().launch_ui()